## Atomics and Reductions

### Learning objectives

In this lab you will learn:

- The difference between transformations and reductions, and how they relate to threading strategy
- Atomic operations and atomic reductions

After completing this lab, you will be able to understand how to implement basic reduction operations.

### Prerequisites

It is assumed that participants have experience with:

- How to launch CUDA kernels that use both blocks and threads


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- Selecting an optimal kernel launch configuration for exposing massive parallelism


- Achieving optimal global memory throughput through coalescing

## Motivating Example

Consider the following code snippet. This is just C code, not CUDA code. We have a large array and we want to get the sum of the array. The obvious way to implement this in serial C code is to initialize a running sum variable `sum` to 0, then use a for loop through the array and add elements one by one.

```
const int size = 100000;
float a[size] = {...};
float sum = 0;
for (int i = 0; i < size; i++) 
    sum += a[i];
```

This problem is a little different from the problems you tackle when you first encounter parallel programming. For example, a typical starting problem is to add one vector to another vector. In that case, the output dataset is the same size as the input dataset. But in this case, the size of the output is just a single number.

### Transformations and Reductions

The case of adding two vectors together can be categorized as part of the general class of problems we will call **transformations**. In a transformation, the size of the output dataset is approximately the same size as the input dataset. In transformations, it's usually straightforward to figure out how to write CUDA code that implements the algorithm optimally. But let's think about this specifically: when we write CUDA code to solve this problem, we need to think about *what each thread should do*. This is part of the process of writing any CUDA algorithm (or any parallel programming algorithm more generally): how can we decompose the problem into multiple parts, with each thread performing one (or multiple) parts? When we write a transformation algorithm, we usually write a kernel from a single thread's perspective. We write the code that one thread should do, and rely on the kernel launch occurring across many threads (and blocks) to parallelize the operation. For a transformation, a common strategy is to assign one thread per output point. A thread's job is just to do whatever work is necessary to compute the correct result for its assigned point in the output. Problems such as vector addition or matrix multiplication naturally lend themselves to such a strategy.

![](images/transformation_vs_reduction.png)

But our example above, where we compute the sum of an array, is a case of the class of problems we will call **reductions**: the size of the output dataset is much smaller than the size of the input dataset. Very often, reductions just produce a single number (a scalar). What would happen if we applied our same strategy of applying one thread per output point? Well, we only have one output point, so we'd only use one thread. Then we'd just write the algorithm serially in CUDA, and our performance would be very poor (thinking back to our previous discussions on how CUDA relies on exposing massive parallelism -- launching lots of threads -- to achieve optimal GPU performance).

### Reduction: Naive Threading Strategy

What if instead of using a strategy of one thread per output point, we use one thread per input point for our sum reduction problem? That is, we would write:

```
*c += a[i];
```

This would allow us to launch a large number of threads, assuming the size of the input dataset is large, which is exactly what we want. Unfortunately, while this kernel is simple to write, this kernel doesn't work: the output in `c` would not be correct.

In order to understand why that's true, let's look at the actual code the GPU executes. (CUDA C++ code is a high-level representation of what we want to do; the compiler has to turn that into instructions the GPU can actually execute.) The GPU code might look like this:

```
LD   R2, &a[i]         (thread independent)
LD   R1, c             (READ)
ADD  R3, R1, R2        (MODIFY)
ST   c,  R3            (WRITE)
```

We need to load the input data point `a[i]` and initial output data `c` into registers (`R2` and `R1`). Then we can add these two together into register `R3`, and then write the output back to `c`.

What happens if many threads are executing this code? Well, each thread would be able to correctly execute the first load instruction, retrieving its assigned value from the array `a`. But as they read and modify `c` we run into trouble, because these threads might be doing this at the same time: recall that CUDA does not enforce any specific order of thread execution. So we might have many threads writing to `c` at the same time (for example), and this is undefined behavior in the CUDA programming model.

### Atomics to the Rescue

To save this example, we're going to take advantage of a special hardware instruction on NVIDIA GPUs: atomics. Atomic operations are exposed in the CUDA C++ programming language. Instead of writing `*c += a[i]`, we write the atomic operation for addition:

```
atomicAdd(c, a[i]);
```

`atomicAdd` takes as arguments the address of the location to be updated (as the first parameter) and the data to be added to it (as the second parameter).

Atomics are a new kind of hardware instruction that replace three other instructions (read, modify, and write) and replace them with an indivisible read-modify-write instruction. For example, the three hardware instructions that perform these operations in the example above might be replaced as follows:

```
LD                    R2, &a[i]
RED.E.ADD.F32.FTZ.RN  c,  R2
```

The fundamental nature of atomic operations is that while a given thread is accessing `c`, *no other thread may also be accessing `c`*. That is what we mean by the operation being indivisible: nothing else can interfere with it.

So now we've replaced the undefined behavior from earlier with well-defined behavior: because only one thread can be modifying `c` at a time, the result should not depend on the order in which threads execute (assuming the operations are associative, which is true for integer operations). But the cost is that we have almost fully serialized the algorithm: threads can only be executing the atomics one at a time. In particular, there is hardware in the L2 cache whose job it is to accept atomic operations and then ensure that they are executed correctly, one at a time, regardless of how many atomic instructions are issued. So after the threads all load the data in from `a`, each thread must get in line and can only perform its read-modify-write operation once the previous thread has completed.

This serialization of the algorithm may have substantial performance implications. We should generally not assume we can achieve full memory throughput when using atomics. We are trading off performance for predictability and correctness.

### Other Atomics

There are other types of atomic operations. We looked at atomic addition above. Here's a longer list:

- `atomicMax/Min`  (choose the maximum or minimum)

- `atomicAdd/Sub`    (add to or subtract from)

- `atomicInc/Dec`    (increment or decrement, accounting for overflow/underflow)

- `atomicExch/CAS`   (swap values, or conditionally swap values)

- `atomicAnd/Or/Xor` (bitwise operations)

Atomics can work on multiple kinds of datatypes (signed/unsigned ints, float, double, etc.), but they are limited on the set of data types they can work on, and this has historically varied by GPU architecture. You can read more about atomic operations in the [CUDA programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#atomic-functions).

## Exercise

The code [exercises/reduction.cu](exercises/reduction.cu) implements a reduction serially on the CPU: we calculate the sum of an array. Refactor it to do the reduction on the GPU, with a kernel that uses atomic operations. Check that it yields the correct result. A solution can be found in [solutions/reduction.cu](solutions/reduction.cu).

In [ ]:
!nvcc -arch=native -o reduction exercises/reduction.cu; ./reduction

Your solution should properly coalesce global memory reads to the array `A` by having consecutive threads access consecutive locations in the array. But what is the performance like for atomics? Let's run your solution through Nsight Compute to find out.

In [ ]:
!ncu --section SpeedOfLight reduction

You probably saw that we only get a very small fraction of achievable memory bandwidth. Vary the number of elements in the array, `N`, and see if that changes.

Next, we will look at how to solve the same reduction problem in a more efficent way using CUB.

## Reduction with CUB

In the last example, we created our own reduction code that utilizes atomics to safely increment a total sum. While we got the correct answer, it is not as efficient as possible. But if a developer had to spend all their time optimizing supplementary routines such as reduction, there would be no time for science! This is the idea behind ***CUB***, a C++ header library that offers high performance, reusable kernel primitives for CUDA. (CUB is included by default in recent versions of CUDA, starting with 11.0. If you are using an earlier version of CUDA, you can obtain it here: https://nvlabs.github.io/cub/download_cub.html)

We will explore CUB in depth in a later module, but for now we will look at an introduction and revisit the exercise we just finished, this time utilizing CUB.

To start using CUB, include the header library in your code: `#include <cub/cub.cuh>`.

Check out the sum reduction we worked through earlier, this time using CUB, in [exercises/reduction_cub.cu](exercises/reduction_cub.cu).

Compile and run the code to verify correctness, then check how its performance compares to the previous implementation.

In [ ]:
!nvcc -arch=native -o reduction_cub exercises/reduction_cub.cu; ./reduction_cub
!ncu --section SpeedOfLight reduction_cub

CUB allows us to achieve high performance kernel level primitives without having to write them ourselves. In a later module, we will learn more about the structure and syntax by walking through examples of a few of the kernel primitives CUB offers.

### Atomic Tips and Tricks

What are some other things we could do with atomics? Well, there's other types of reductions than the summation example we've already looked at. We can extend that example to computing the maximum (or minimum) of all the elements in an array, which is also a reduction problem that atomics may be useful for. But there are examples that are not immediately as obvious.

For example, we could use atomics to manage a queue. We would have a variable that represents the next item in a queue, and any thread that wanted the next item in the queue would use an atomic operation. Now, we didn't address this above, but most atomic operations have a return value as well, which is the old value prior to the atomic update:
```
int my_position = atomicAdd(order, 1);
```
If we do this atomic add operation, what will happen is that this thread gets assigned the item in location `my_position` to work on, and any thread that comes along later will get some other value (the next location in the queue).

Another example is that we have some buffer (allocated area) in memory and we're dealing with an algorithm that produces a variable amount of data per thread. How can we collect all of this in parallel in a single buffer? 

Let's say our algorithm is looking at a large array for matches to a particular value. When we have a match, we want to record the index of that match in a (presumably) smaller buffer that records the indices where that value is found. Since we want to do this in a thread parallel fashion, we have each thread work on a separate location, and each thread may produce zero, one, or more matches, and need to write those locations (if any exist) to the output buffer. This may seem challenging to a beginning CUDA programmer. Of course, there's a naive implementation for this particular problem: we could just allocate an array the same size as the input array, since that is the upper bound on the number of matches we could have. However, that would still leave us with questions, like how do we get rid of the empty spaces afterward?

![](images/reserve_buffer_space.png)

One way we could do this with this buffer is to have each thread create a local buffer of data corresponding to its set of matches. When that thread wants to write its data to the global buffer, we'll use atomicAdd to reserve our space. But now instead of adding just one entry, we're going to add a number of entries corresponding to the number of entries our thread found (in this code example, that would be `my_dsize`).

```
int my_dsize = var;

float local_buffer[my_dsize] = {...};

int my_offset = atomicAdd(buffer_idx, my_dsize); 

// buffer_ptr + my_offset now points to the first reserved location (which has length my_dsize)

memcpy(buffer_ptr + my_offset, local_buffer, my_dsize * sizeof(float));
```

In this way we can use atomics to do this algorithm, sometimes known as **stream compaction**, which is a fairly simple algorithm when written serially but requires some care when implemented in parallel. Atomics are not necessarily the best (highest performing) method, but they are one way to go.

## Summary

In this lab you have learned:

- The difference between transformations and reductions

- How to implement parallel reductions using atomic operations

You should now be able to tackle a broader range of computation problems, and hopefully have a broader set of tools for thinking about how to write a parallel algorithm in CUDA.

## Further Study

[Parallel reduction](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf)

[Floating point](https://developer.nvidia.com/sites/default/files/akamai/cuda/files/NVIDIA-CUDA-Floating-Point.pdf)

## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.